This is an example of the basic interface to control and get information from the robot.
The basic robot interface basically just inherits ur_rtde intefaces.
There is a lot more in that interface than what is shown here, it can be found in the ur_rtde documentation:
https://sdurobotics.gitlab.io/ur_rtde/api/api.html

# Robot Controller
To create a robot controller instance, we need the robot's information first: it's IP address.
it is saved in the robot metadata:


In [1]:
from ur_lab.robot_inteface.robots_metadata import ur5e_1
print("Robot metadata: ", ur5e_1)

Robot metadata:  {'name': 'ur5e_1', 'ip': '192.168.0.10'}


Create instance of robot controller and move the robot to the home configuration:

In [2]:
from ur_lab.robot_inteface.robot_interface import RobotInterface

robot = RobotInterface(ur5e_1["ip"])
robot.move_home()

Commands in this tutorial, such as move_home and others are not considering collisions (no motion planning). If you apply them and there is a collision or the robot is in a singular configuration, the robot will stop and you will have to manually move it back to a safe configuration.
If the robot collided in the last cell, just move it manually to the home config before you continue.

# Joint Space

The robot configuration is a vector of the angles (in radians) of all it's joints. There are 6 joints that can be between [-2pi, 2pi].
However, sometimes it is recommended to limit to [-pi, pi], as some joints cant reach the full range when the
robot is on a table, and reducing the range reduces the configuration space when motion planning. Also, when some joints reach the edges, cables can get wrapped around the robot and make it stop.

get current robot configuration, which is the home configuration:


In [5]:
q = robot.getActualQ()
print("robot configuration at home: ", q)

robot configuration at home:  [1.1920928955078125e-06, -1.5708047352232875, 2.843538393193512e-05, -1.570781666275554, -5.547200338185121e-06, 1.0764599210233428e-05]


move it in each joint separately, by 10 degrees, then move back home (all joints at once):

In [4]:
import time

new_q = q.copy()
for i in range(6):
    new_q[i] += 20 * 3.14 / 180
    robot.moveJ(new_q, speed=0.3, acceleration=0.3)
    time.sleep(0.5)

In [5]:
robot.moveJ(q, speed=1, acceleration=1)

True

Note the speed and acceleration, this is the speed of the joint in rad/s, and the acceleration in rad/s^2.
It is recommended to do your tests with low speeds (~0.3) and when everything works safely you can increase.

## Move Path

There is an option to move the robot through a path of configurations, this is useful for complex motions.
It can simply be done by passing a list of configurations to the move_path function.


In [7]:
import numpy as np

# make sure robot at home first:
robot.move_home()

q_path = [[-np.pi/10, -np.pi/2 -np.pi/8, np.pi/4, -np.pi/2, 0, 0],
          [np.pi/10, -np.pi/2  -np.pi/6, np.pi/4, -np.pi/2, np.pi/5, 0],
          [np.pi/8, -np.pi/2 , np.pi/4, -np.pi/2, np.pi/5, np.pi/5],
          q]

robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.0)

Note how the robot moves to each config in the path, then slows down to a stop before moving to the next one.
If we want that movement to be continuous, which mean the robot will not slow to a complete stop before moving to the next configuration on the path, but will blend the movement between the configurations, we can set the blend_radius to a value greater than 0. This will make the robot move smoothly between the configurations.

This should look smoother and be faster since the robot doesn't reach a complete stop between the configurations:

In [9]:
robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.1)

## Move Asynchronously

Movement commands can be asynchronous, which means that the command will return immediately and the robot will start moving in the background while you can run code concurrently. This can be doen by passing the asynchronous=True argument to movement functions.
If another command is sent to the robot while it is moving, the robot will stop the current movement and start the new one.

In [14]:
robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.05, asynchronous=True)
time.sleep(0.5)
print("this is being printed while the robot is moving")
print("joint velocities:", robot.getActualQd())
time.sleep(5)
print("stopping robot in the middle of the path")
robot.stopJ()

robot.move_home()

this is being printed while the robot is moving
joint velocities: [-0.0026534826029092073, -0.13892582058906555, 0.03717377781867981, -0.0, 0.029471246525645256, -0.14168062806129456]
stopping robot in the middle of the path


# Cartesian Space

In [16]:
robot.move_home()

pose = robot.getActualTCPPose()
print("robot pose at home: ", pose)

robot pose at home:  [-0.002555864064543423, -0.37057366730368074, 1.0809860021668733, -0.007722413265855482, -2.214470623617691, 2.221465386183995]
